## <span style=color:blue>Working with BestiaPop</span>

### <span style=color:blue>I downloaded BestiaPop from github: https://github.com/JJguri/bestiapop .  That was built using python 3.7.  It doesn't work with python 3.10 because the file nasapower_connector.py within BestiaPop uses the dataframe append command, but in python 3.10 pandas was updated and append was deprecated.  (One has to use concat instead).  So I created a python environment that runs python 3.7 rather than python 3.10.  I run this notebook in my python 3.7 environment.</span>

In [5]:
import sys
import os
import subprocess
import shutil
# use shutil.rmtree(<path_to_folder>) to do an rm -r on the folder

import json
import csv
# import yaml

import pandas as pd
import numpy as np

# import matplotlib as mpl
# from cycler import cycler

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set(style="darkgrid")

# This useful if I want to give unique names to directories or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime

print('Initial imports successful')

Initial imports successful


<span style=color:blue>
The following cell illustrates how we can invoke methods directly from the github version of bestiapop.  However, since the bestiapop code uses an append on a dataframe, and pandas deprecated the append operator for dataframes, I could not make the downloaded bestiapop work.

More specifically, even though I am running in python3.7, if I use BestiaPop from the github repo then somehow my python kernel finds a recent version of pandas in which dataframe append has been deprecated.  (See the last cell in this notebook.)  

So I made a one-line change to the module nasapower_connector.py within the file bestiapop/bestiapop/connectors/nasapower_connector.py.  Specifically, somewhere near line 353 I replaced the line "total_climate_df = total_climate_df.append(var_year_lat_lon_df)" with the line "total_climate_df = pd.concat([total_climate_df, var_year_lat_lon_df])".

Also, it seems that I needed to change the names of both the outer "bestiapop" directory and the inner "bestiapop" directory, because otherwise the system was giving me errors because it was calling some stray files from a previous import of the PyPi version of bestiapop.</span>


In [7]:
# imitating example in 
# https://github.com/JJguri/bestiapop/blob/master/sample-data/ExampleMapsTasmania.ipynb

sys.path.append('/Users/rick/github/bestiapop/')
from bestiapop import bestiapop as bp


# sys.path.append('/Users/rick/github/bestiapop_WORKING/')
# sys.path.append('/Users/rick/github/bestiapop_WORKING/bestiapop_INSIDE')
# from bestiapop_INSIDE import bestiapop as bp

working_dir = '/Users/rick/AG-CODE-BESTIAPOP/OUTPUTS/'

target_dir = 'test_' + curr_timestamp()
output_path = working_dir + target_dir
print('Output files will be written into: ', output_path, '\n')

os.mkdir(working_dir + target_dir)

year_range = '2010-2012' # using this string format rather than, e.g., np.array([2010,2011,2012])

# these are the only climate variables supported by BestiaPop on NASAPOWER data
climate_variables = np.array(['daily_rain', 'max_temp', 'min_temp', 'radiation'])

# on NASAPOWER, it works on round numbers and xx.5 numbers only; 
# if you give it a coordinate such as 145.63 it will round to 145.5 and/or 146
lat_range = [-42.5, -41.5] 
lon_range = [144.5, 146] 

action = 'generate-climate-file'
data_source = 'nasapower'
output_type = 'met'

print('Fetching data for {}/{}'.format(lat_range, lon_range))
climate_data = bp.CLIMATEBEAST(
        action=action,
        data_source=data_source,
        output_path=output_path,
        output_type=output_type,
        input_path=None,
        climate_variables=climate_variables,
        year_range=year_range,
        lat_range=lat_range,
        lon_range=lon_range,
        multiprocessing=None
        )
print()
print('The climate_data object has value:\n', str(climate_data))
print('\n\nThe fields with the climate_data object are:\n\n', climate_data.__dir__())
print()
print('invoking climate_data.process_records(action)\n')
climate_data.process_records(action)
print('\nfinished with invocation of climate_data.process_records(action)\n\n')

file_list = subprocess.run(["ls -l " + output_path], shell=True, capture_output=True, text=True)
print('The list of files created is as follows:\n')
print(file_list.stdout)


Output files will be written into:  /Users/rick/AG-CODE-BESTIAPOP/OUTPUTS/test_2023-05-02_18-14-38 

Fetching data for [-42.5, -41.5]/[144.5, 146]

The climate_data object has value:


The fields with the climate_data object are:

 ['logger', 'multiprocessing', 'total_parallel_climate_df', 'final_parallel_lon_range', 'output_type', 'action', 'climate_variables', 'data_source', 'input_path', 'outputdir', 'year_range', 'lat_range', 'lon_range', '__module__', '__doc__', '__init__', 'process_parallel_records', 'process_parallel_met', 'process_parallel_output', 'process_records', '__dict__', '__weakref__', '__repr__', '__hash__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__eq__', '__ne__', '__gt__', '__ge__', '__new__', '__reduce_ex__', '__reduce__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']

invoking climate_data.process_records(action)

Total Progress: 100%|#############################| 3/3 [00:37<